<a href="https://colab.research.google.com/github/kazim-glitch/Deep-learning/blob/main/Deep_convulotional_generative_adversarial_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch.utils.tensorboard import  SummaryWriter

In [ ]:
class discriminator(nn.Module):
  def __init__(self,channels_image,features):
    super(discriminator,self).__init__()
    self.disc = nn.Sequential(
        
        nn.Conv2d(
         channels_image,features,kernel_size=4,stride=2,padding=1   
        ),
        nn.LeakyReLU(0.2),
        self.block(features,features*2,4,2,1), 
        self.block(features*2,features*4,4,2,1),
        self.block(features*4,features*8,4,2,1),
        nn.Conv2d(features*8,1,kernel_size=4,stride=2,padding=0),
        nn.Sigmoid(),
    )

  def block(self,input_chanel,output_chanel,kernel_size,stride,padding):
    return nn.Sequential(
        nn.Conv2d(
            input_chanel,
            output_chanel,
            kernel_size,
            stride,
            padding,
            bias=False

        ),
        nn.BatchNorm2d(output_chanel),
        nn.LeakyReLU(0.2), # as mentioned in the paper batchnorm can only be used in the final layers of the disc
        # nn.Tanh(),
        # nn.ReLU(),

    )
  def forward(self,x):
    return self.disc(x)

class generator(nn.Module):
  def __init__(self,noise_z_dim,channels_image,feature):
    super(generator,self).__init__()
    self.gen = nn.Sequential(
        self.block(noise_z_dim,feature*16,4,1,0),
        self.block(feature*16,feature*8,4,2,1),
        self.block(feature*8,feature*4,4,2,1),
        self.block(feature*4,feature*2,4,2,1),
        nn.ConvTranspose2d(
            #upscaling the image
            feature*2,channels_image,kernel_size=4,stride=2,padding=1,
        ),
        nn.Tanh(),
        
        

    )

  def block(self,inp_chanel,out_chanel,kernel_size,stride,padding):

    return nn.Sequential(
     nn.ConvTranspose2d(
         inp_chanel,
         out_chanel,
         kernel_size,
         stride,
         padding,
         bias=False

         
     ),
     nn.BatchNorm2d(out_chanel),
     nn.ReLU() 

     )
  def forward(self,x):
    return self.gen(x)  

     

def initiliaze_weights(model):
  for m in model.modules():
    if isinstance(m, (nn.Conv2d,nn.ConvTranspose2d,nn.BatchNorm2d)):
      #initializing weights for discriminator
      nn.init.normal_(m.weight.data,0.0,0.02) #in paper normal distribution 0 and standard deviation =0.02

    # if isinstance(m, nn.ConvTranspose2d):
    #   nn.init.normal_(m.weigth.data,0.0,0.02)  

    # if isinstance(m, nn.BatchNorm2d):
    #   nn.init.normal_(m.weigth.data,0.0,0.02)  





In [ ]:
def forward_check():
  N,in_chanel,H,W = 8,3,64,64
  noise_z = 100

  x=torch.randn((N,in_chanel,H,W))

  disc = discriminator(in_chanel,8)
  initiliaze_weights(disc)
  assert disc(x).shape == (N,1,1,1)

  gen = generator(noise_z,in_chanel,8)
  initiliaze_weights(gen)
  z =torch.randn((N,noise_z,1,1))
  assert gen(z).shape == (N,in_chanel,H,W)

forward_check()